In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append("..")
from simdb.databaseModel import *
from simdb.databaseAPI import *

In [3]:
# db_path = 'test_WORKS.db'
db_path = '../examples/setup_databases/andrej_raw.db'

In [ ]:
from sqlalchemy import or_, and_

## For migration to databaseAPI:

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [ ]:
class DatabaseConnection(object):
    
    def __init__(self, db_path):
        self.db_path = db_path
        self.session = None
        

    def __enter__(self):
        # make a database connection and return it
        if not os.path.exists(self.db_path):
            raise "%s does not exist."
        
        engine = create_engine('sqlite:///{}'.format(self.db_path))
        Session = sessionmaker()
        self.session = Session(bind=engine)
        
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.session.close()

In [ ]:

class SQLAlchemyDBConnection(object):
    """SQLAlchemy database connection"""

    def __init__(self, connection_string):
        self.connection_string = connection_string
        self.session = None

    def __enter__(self):
        engine = create_engine(self.connection_string)
        Session = sessionmaker()
        self.session = Session(bind=engine)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.session.close()

In [ ]:
with DatabaseConnection(db_path) as db:
    print db.session.query(Keywords)

In [ ]:
q = s.session.query(Keywords)
[e.name for e in q.filter(Keywords.value != None).all()]

In [ ]:
s.close()

In [ ]:
s.add(Main())

In [ ]:
s.commit()

In [ ]:
s.close()

## HowTo

### Load Entry Table

In [ ]:
table = getEntryTable(db_path, load_keys=True, load_tags=True) # load whole table

In [ ]:
table = get_entry_table(db_path)

In [ ]:
table

### Obtain data for certain entry

In [4]:
entry_id = "2017_08_01_2ub_k27_02_06"

In [ ]:
details = get_entry_details(db_path, entry_id)
details

### Select entries

In [ ]:
table[selectByTag(table, "test")] # select by tag

In [ ]:
table[selectByKeyword(table, "ff", "MARTINI")] # select by keyword

In [ ]:
# combinations of selections
ff       = selectByKeyword(table, "ff", "MARTINI")
chainlen = selectByKeyword(table, "chainlen", "1")
tags     = selectByTag(table, "test2")

table[ff & chainlen & tags]

In [ ]:
# another example
ff       = selectByKeyword(table, "ff", "MARTINI")
chainlen = selectByKeyword(table, "chainlen", "2")
linker1  = selectByKeyword(table, "linker", "k48")
linker2  = selectByKeyword(table, "linker", "k63")
number   = selectByKeyword(table, "number", "01")

table[ff & chainlen & (linker1 | linker2) & ~number]

### Working with groups

In [ ]:
# add some entries to a group
sel = table[selectByKeyword(table, "linker", "k48")]
[add_group(db_path, entry_id, "Group 1") for entry_id in sel.entry_id]

# a second group
sel = table[selectByKeyword(table, "ff", "MARTINI")]
[add_group(db_path, entry_id, "Group 2") for entry_id in sel.entry_id]

In [ ]:
# get entries of group 1
entry_group_table1 = get_entry_table(db_path, group_names=["Group 1"], columns=["entry_id", "path"])
print("{} entries loaded.".format(entry_group_table1.shape[0]))

# get entries of group 2
entry_group_table2 = get_entry_table(db_path, group_names=["Group 2"], columns=["entry_id", "path"])
print("{} entries loaded.".format(entry_group_table2.shape[0]))

In [ ]:
# get entries from both groups
# some entries might be in both groups
merged_table = get_entry_table(db_path, group_names=["Group 1", "Group 2"], columns=["entry_id", "path"])
print("Merge results in {} entries.".format(merged_table.shape[0]))

In [ ]:
# remove some etnries from Group 1
sel = table[selectByKeyword(table, "ff", "MARTINI")]
[remove_group(db_path, entry_id, "Group 1") for entry_id in sel.entry_id]

### Usefull for backend / viewer

In [ ]:
# open database (in SQLAlchemy we open a session)
# s = openDatabase(db_path) 
# one should not keep the DB open

# better:
with openDatabase(db_path) as db:
    print db
    

In [ ]:
get_tags(db_path)  # get all tags which are used in DB

In [ ]:
keywords = get_keywords(db_path)  # get all keyword with the values they have
pd.Series(keywords) 

In [ ]:
get_groups(db_path)

## Add, edit remove Metadata

In [14]:
s = open_database(db_path)

In [15]:
entry = s.query(Main).filter(Main.entry_id == entry_id).first()
# entry.keywords

In [16]:
entry.meta.all()

[MetaGroups(main_id='117', name='barostat'),
 MetaGroups(main_id='117', name='thermostat'),
 MetaGroups(main_id='117', name='More Information')]

In [ ]:
mg = entry.meta.filter_by(name="barostat").first()

In [ ]:
mg.entries.all()

In [ ]:
entry.groups

In [ ]:
s.close()

### Tags

In [ ]:
add_tag(db_path, entry_id, "my_tag")

In [ ]:
remove_tag(db_path, entry_id, "my_tag")

### Keywords

In [ ]:
# add two keywords to certain entry
add_keyword(db_path, entry_id, keyword1="bla", keyword2="blub")

In [ ]:
# alter keyword1 of entry
alter_keyword(db_path, entry_id, keyword1="my_value")

In [ ]:
# remove keyword2 if it is "blub" and force remove keyword1
remove_keyword(db_path, entry_id, keyword1=None, keyword2="blub")

### Meta information

In [11]:
meta_group_name = "More Information"

In [12]:
# add some information to entry
add_meta_group(db_path, entry_id, meta_group_name)
add_meta_data(db_path, entry_id, meta_group_name, 
              run="the best one", 
              importance="rather large", 
              version="quite new")

True

In [13]:
alter_meta_data(db_path, entry_id, meta_group_name,
               run = "actually not that good")

True

In [20]:
# remove some meta data entries from group
remove_meta_data(db_path, entry_id, meta_group_name,
                run=None,
                version="quite new")

# or remove the whole group
remove_meta_group(db_path, entry_id, meta_group_name)

True

In [21]:
# check out changes
pd.DataFrame(get_entry_details(db_path, entry_id)['meta'])

,barostat,thermostat
T_rel,NaN,1
T_target,NaN,300
p_compressibility,0.0003,NaN
p_coupling,iso,NaN
p_rel,12,NaN
p_target,1,NaN
type,Parrinello-Rahman,v-rescale


### Running functions

## Deva

* Add tags/keywords to all simulations in pandas table
* Add simulations to DB from pandas table